In [10]:
%run Utils.ipynb

In [11]:
language = 'Greek'

In [12]:
originalLanguage = ''
translatedLanguage = ''

if language.lower() == 'greek':
    originalLanguage = 'Greek'
    translatedLanguage = 'English'
elif language.lower() == 'english':
    originalLanguage = 'English'
    translatedLanguage = 'Greek'

In [13]:
# Read the tables.json
tableFileName = fr'DevSplit\{translatedLanguage}\tables.json'
tablesData = loadJson(tableFileName)
tablesData[0].keys()

dict_keys(['column_names', 'column_names_original', 'column_types', 'db_id', 'foreign_keys', 'primary_keys', 'table_names', 'table_names_original'])

In [14]:
# Read the dev.json
devFileName = fr'DevSplit\{originalLanguage}\dev.json'
devDataset = loadJson(devFileName)
devDataset[0].keys()

dict_keys(['db_id', 'query', 'query_toks', 'query_toks_no_value', 'question', 'question_toks', 'sql'])

In [15]:
dictWithTablesData = {}

# Create a dictionary where db_id is the key and the corresponding jsonSchema is the value
for jsonSchema in tablesData:
    dictWithTablesData[jsonSchema['db_id']] = jsonSchema

In [16]:
systemContent = f'I will give you {originalLanguage} questions and you will translate it to {translatedLanguage}. Please return only the translated question.'

totalNumOfQuestions = len(devDataset)
percent = int(totalNumOfQuestions * 0.1)  # Calculate 1% of the total tables

# Iterate over each questionInfo in devDataset
for index, questionInfo in enumerate(devDataset, start =1):
    # Extract the db_id from the questionInfo
    dbId = questionInfo['db_id']

    # Retrieve table and column information for the current dbId
    listWithTableAndColumnInfo = returnTableAndColumnInfoForDbId(dictWithTablesData, dbId, columnNameAttribute = 'column_names')
    dictWithSchemaInfo = createSchemaInfoDict(listWithTableAndColumnInfo)

    # Extract the original question from questionInfo and append it to listWithOriginalQuestionPerDbId
    question = questionInfo['question']
    
    prompt = createTheDesiredPromtForQuestionsTranslation(question, dictWithSchemaInfo, language=translatedLanguage)
    # Translate the question 
    questionTranslated = removeTextInParentheses(gptTranslateInfo(systemContent, prompt))

    # tokenize the question
    tokenizedQuestion = tokenizeSentence(questionTranslated)

    # Update questionInfo with the translated question and its tokenized form
    questionInfo['question'] = questionTranslated
    questionInfo['question_toks'] = tokenizedQuestion
    
    if index % percent == 0:
        print(f"{index}/{totalNumOfQuestions} of questions have been examined.")

11/110 of questions have been examined.
22/110 of questions have been examined.
33/110 of questions have been examined.
44/110 of questions have been examined.
55/110 of questions have been examined.
66/110 of questions have been examined.
77/110 of questions have been examined.
88/110 of questions have been examined.
99/110 of questions have been examined.
110/110 of questions have been examined.


In [17]:
#Save the updated dev.json
writeJsonToFile(devDataset, f'MachineTranslated\{translatedLanguage}\dev.json')

JSON data written to MachineTranslated\English\dev.json
